In [1]:
import sys
import shutil
sys.path.append('../dependencies/')
import pyemu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import pathlib as pl
import os, platform

## set some global vars we need

In [2]:
wkdir = pl.Path('.')
template_dir = pl.Path('./tmpdir')
if template_dir.exists():
    shutil.rmtree(template_dir)
template_dir.mkdir()
    
input_yml = 'TestExample.yml'
with open(wkdir / input_yml, 'r') as ifp:
    indat = yaml.safe_load(ifp)

# Set up `tpl` file for parameterization
### parameterize T and S globally

In [3]:
T_init = indat['project_properties']['T']
S_init = indat['project_properties']['S']

indat['project_properties']['T'] = f'~{"global_T":^16s}~'
indat['project_properties']['S'] = f'~{"global_S":^16s}~'


### get the starting apportionment values and parameterize them as well

In [4]:
well_keys = [i for i in indat.keys() if i.startswith('well_')]
app_keys = [[j for j in indat[i].keys() if j.startswith('stream_apportionment')]
                     for i in well_keys]
pending_wells = [i for i in well_keys if 'pending' in indat[i]['status']]
allkeys = dict(zip(well_keys, app_keys))
allkeys

{'well_92743': ['stream_apportionment158'],
 'well_73262': ['stream_apportionment168'],
 'well_93832': ['stream_apportionment159'],
 'well_23983': ['stream_apportionment43'],
 'well_70548': ['stream_apportionment82'],
 'well_70262': ['stream_apportionment138'],
 'well_67401': ['stream_apportionment135'],
 'well_986': ['stream_apportionment69'],
 'well_68690': ['stream_apportionment188'],
 'well_23883': ['stream_apportionment49'],
 'well_72737': ['stream_apportionment189'],
 'well_24155': ['stream_apportionment77'],
 'well_24037': ['stream_apportionment51'],
 'well_23978': ['stream_apportionment0'],
 'well_91803': ['stream_apportionment182'],
 'well_91706': ['stream_apportionment161'],
 'well_91411': ['stream_apportionment108'],
 'well_92199': ['stream_apportionment191'],
 'well_92045': ['stream_apportionment115'],
 'well_93141': ['stream_apportionment78'],
 'well_91562': ['stream_apportionment166'],
 'well_73744': ['stream_apportionment74'],
 'well_92299': ['stream_apportionment157'],


In [5]:
pending_wells

['well_93832']

In [6]:
pars = list()
parvals = list()
for k,v in allkeys.items():
    for cv in v:
        cpar = f'{k}__{cv}'
        pars.append(cpar)
        parvals.append(indat[k][cv]['apportionment'])
        indat[k][cv]['apportionment'] = f'~{cpar:^45}~'

In [7]:
pars_df = pd.DataFrame(index = pars, data = {'parval1':parvals})

In [8]:
pars_df = pd.concat([pars_df, pd.DataFrame(index = ['global_s','global_t'], data = {'parval1':[S_init,T_init]})], ignore_index=True)

In [9]:
pars_df

,parval1
0,0.222260
1,0.144235
2,0.160961
3,0.705632
4,0.900000
...,...
201,0.400670
202,0.391291
203,0.852820
204,0.150000


In [10]:
with open(template_dir / f"{input_yml}.tpl",'w') as ofp:
    ofp.write('ptf ~\n')
    documents = yaml.dump(indat, ofp, default_flow_style = False, sort_keys= False)

# make `ins` file and external forward run file

In [11]:
basedeplobs = [f"{indat[k]['name']}:bdpl" for k in indat.keys() if 'stream' in k]

In [12]:
[i for i in basedeplobs if '93832' in i]

['TomorrowRiver:93832:bdpl']

In [13]:
unique_rivers = list(set([i.split(':')[0] for i in basedeplobs]))

In [14]:
unique_rivers

['TomorrowRiver']

In [15]:
# pending well already included
# basedeplobs.extend([f"{i}:{j.replace('well_','')}:bdpl" for i in unique_rivers for j in pending_wells])

In [16]:
basedeplobs.extend([f'{i}:{j}:bdpl' for i in unique_rivers for j in ['total_proposed','total_existing','total_combined']])

In [17]:
with open(template_dir / 'basedeplobs.dat', 'w') as ofp:
    [ofp.write(i + '\n') for i in basedeplobs]

# make forward run script external file

In [18]:
output_ts = ['TomorrowRiver:92696','TomorrowRiver:70974']
times = range(365*4,365*5+1)

In [19]:
ts_obs = []
for c_ts in output_ts:
    ts_obs.extend([f'{c_ts}__{i}' for i in times])

In [20]:
ts_obs

['TomorrowRiver:92696__1460',
 'TomorrowRiver:92696__1461',
 'TomorrowRiver:92696__1462',
 'TomorrowRiver:92696__1463',
 'TomorrowRiver:92696__1464',
 'TomorrowRiver:92696__1465',
 'TomorrowRiver:92696__1466',
 'TomorrowRiver:92696__1467',
 'TomorrowRiver:92696__1468',
 'TomorrowRiver:92696__1469',
 'TomorrowRiver:92696__1470',
 'TomorrowRiver:92696__1471',
 'TomorrowRiver:92696__1472',
 'TomorrowRiver:92696__1473',
 'TomorrowRiver:92696__1474',
 'TomorrowRiver:92696__1475',
 'TomorrowRiver:92696__1476',
 'TomorrowRiver:92696__1477',
 'TomorrowRiver:92696__1478',
 'TomorrowRiver:92696__1479',
 'TomorrowRiver:92696__1480',
 'TomorrowRiver:92696__1481',
 'TomorrowRiver:92696__1482',
 'TomorrowRiver:92696__1483',
 'TomorrowRiver:92696__1484',
 'TomorrowRiver:92696__1485',
 'TomorrowRiver:92696__1486',
 'TomorrowRiver:92696__1487',
 'TomorrowRiver:92696__1488',
 'TomorrowRiver:92696__1489',
 'TomorrowRiver:92696__1490',
 'TomorrowRiver:92696__1491',
 'TomorrowRiver:92696__1492',
 'Tomorrow

In [21]:
allobs = basedeplobs + ts_obs

In [22]:
allobs

['TomorrowRiver:23978:bdpl',
 'TomorrowRiver:24021:bdpl',
 'TomorrowRiver:24050:bdpl',
 'TomorrowRiver:24302:bdpl',
 'TomorrowRiver:70519:bdpl',
 'TomorrowRiver:70520:bdpl',
 'TomorrowRiver:23873:bdpl',
 'TomorrowRiver:23924:bdpl',
 'TomorrowRiver:24087:bdpl',
 'TomorrowRiver:91175:bdpl',
 'TomorrowRiver:73625:bdpl',
 'TomorrowRiver:72183:bdpl',
 'TomorrowRiver:91181:bdpl',
 'TomorrowRiver:92036:bdpl',
 'TomorrowRiver:1302:bdpl',
 'TomorrowRiver:23635:bdpl',
 'TomorrowRiver:23647:bdpl',
 'TomorrowRiver:23648:bdpl',
 'TomorrowRiver:23699:bdpl',
 'TomorrowRiver:23754:bdpl',
 'TomorrowRiver:24216:bdpl',
 'TomorrowRiver:24285:bdpl',
 'TomorrowRiver:24286:bdpl',
 'TomorrowRiver:24294:bdpl',
 'TomorrowRiver:92981:bdpl',
 'TomorrowRiver:23700:bdpl',
 'TomorrowRiver:23867:bdpl',
 'TomorrowRiver:23868:bdpl',
 'TomorrowRiver:23869:bdpl',
 'TomorrowRiver:24169:bdpl',
 'TomorrowRiver:3473:bdpl',
 'TomorrowRiver:23637:bdpl',
 'TomorrowRiver:23716:bdpl',
 'TomorrowRiver:23870:bdpl',
 'TomorrowRiver:

In [23]:
with open(template_dir / 'ts_obs.dat' , 'w') as ofp:
    [ofp.write(c_ts + '\n') for c_ts in output_ts]

In [24]:
times

range(1460, 1826)

In [25]:
indat

{'project_properties': {'name': 'TestExample',
  'T': '~    global_T    ~',
  'S': '~    global_S    ~',
  'default_dd_days': 90.0,
  'default_depletion_years': 5.0,
  'default_pumping_days': 90.0},
 'well_92743': {'name': '92743',
  'status': 'existing',
  'loc': {'x': 89.301241, 'y': 44.463364},
  'Q': 69.0,
  'pumping_days': 90,
  'stream_apportionment158': {'name': 'TomorrowRiver:92743',
   'apportionment': '~     well_92743__stream_apportionment158     ~'},
  'stream_response': ['TomorrowRiver:92743'],
  'dd_response': ['LakeEmily']},
 'well_73262': {'name': '73262',
  'status': 'existing',
  'loc': {'x': 89.290945, 'y': 44.4615},
  'Q': 52.0,
  'pumping_days': 90,
  'stream_apportionment168': {'name': 'TomorrowRiver:73262',
   'apportionment': '~     well_73262__stream_apportionment168     ~'},
  'stream_response': ['TomorrowRiver:73262'],
  'dd_response': []},
 'well_93832': {'name': '93832',
  'status': 'pending',
  'loc': {'x': 89.298, 'y': 44.465700000000005},
  'Q': 485.0,
 

In [26]:
odir = pl.Path('output')

In [27]:
base_data = pd.read_csv(odir/f'{input_yml.replace(".yml","")}.table_report.base_stream_depletion.csv', index_col=0)

In [28]:
base_data

,TomorrowRiver
93832,0.072980
92743,0.009860
73262,0.008878
23983,0.042601
70548,0.477394
...,...
68219,0.000000
90159,0.000000
total_proposed,0.072980
total_existing,2.974664


In [29]:
bdplobs = pd.read_csv(template_dir/'basedeplobs.dat', header=None)
bdplobs.columns = ['obsname']
bdplobs.index = bdplobs.obsname

In [30]:
bdplobs['obs_values'] = np.nan
bdplobs

,obsname,obs_values
obsname,,
TomorrowRiver:23978:bdpl,TomorrowRiver:23978:bdpl,NaN
TomorrowRiver:24021:bdpl,TomorrowRiver:24021:bdpl,NaN
TomorrowRiver:24050:bdpl,TomorrowRiver:24050:bdpl,NaN
TomorrowRiver:24302:bdpl,TomorrowRiver:24302:bdpl,NaN
TomorrowRiver:70519:bdpl,TomorrowRiver:70519:bdpl,NaN
...,...,...
TomorrowRiver:815:bdpl,TomorrowRiver:815:bdpl,NaN
TomorrowRiver:71960:bdpl,TomorrowRiver:71960:bdpl,NaN
TomorrowRiver:total_proposed:bdpl,TomorrowRiver:total_proposed:bdpl,NaN


In [31]:
for cob in bdplobs.obsname:
    riv,wel,_ = cob.split(':')
    print(cob)
    bdplobs.loc[cob, 'obs_values'] = base_data.loc[wel][riv]

TomorrowRiver:23978:bdpl
TomorrowRiver:24021:bdpl
TomorrowRiver:24050:bdpl
TomorrowRiver:24302:bdpl
TomorrowRiver:70519:bdpl
TomorrowRiver:70520:bdpl
TomorrowRiver:23873:bdpl
TomorrowRiver:23924:bdpl
TomorrowRiver:24087:bdpl
TomorrowRiver:91175:bdpl
TomorrowRiver:73625:bdpl
TomorrowRiver:72183:bdpl
TomorrowRiver:91181:bdpl
TomorrowRiver:92036:bdpl
TomorrowRiver:1302:bdpl
TomorrowRiver:23635:bdpl
TomorrowRiver:23647:bdpl
TomorrowRiver:23648:bdpl
TomorrowRiver:23699:bdpl
TomorrowRiver:23754:bdpl
TomorrowRiver:24216:bdpl
TomorrowRiver:24285:bdpl
TomorrowRiver:24286:bdpl
TomorrowRiver:24294:bdpl
TomorrowRiver:92981:bdpl
TomorrowRiver:23700:bdpl
TomorrowRiver:23867:bdpl
TomorrowRiver:23868:bdpl
TomorrowRiver:23869:bdpl
TomorrowRiver:24169:bdpl
TomorrowRiver:3473:bdpl
TomorrowRiver:23637:bdpl
TomorrowRiver:23716:bdpl
TomorrowRiver:23870:bdpl
TomorrowRiver:23926:bdpl
TomorrowRiver:23979:bdpl
TomorrowRiver:24066:bdpl
TomorrowRiver:24080:bdpl
TomorrowRiver:70191:bdpl
TomorrowRiver:1860:bdpl
Tom

In [32]:
bdplobs

,obsname,obs_values
obsname,,
TomorrowRiver:23978:bdpl,TomorrowRiver:23978:bdpl,0.012656
TomorrowRiver:24021:bdpl,TomorrowRiver:24021:bdpl,0.002488
TomorrowRiver:24050:bdpl,TomorrowRiver:24050:bdpl,0.024583
TomorrowRiver:24302:bdpl,TomorrowRiver:24302:bdpl,0.002240
TomorrowRiver:70519:bdpl,TomorrowRiver:70519:bdpl,0.004243
...,...,...
TomorrowRiver:815:bdpl,TomorrowRiver:815:bdpl,0.082662
TomorrowRiver:71960:bdpl,TomorrowRiver:71960:bdpl,0.033554
TomorrowRiver:total_proposed:bdpl,TomorrowRiver:total_proposed:bdpl,0.072980


In [33]:
ts_data = pd.read_csv(odir/f'{input_yml.replace(".yml","")}.table_report.all_ts.csv', index_col=0) 

C:\Users\pruita\Miniconda3\lib\site-packages\pandas\core\algorithms.py:514: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [34]:
ts_data.columns = ts_data.columns.str.split('__').str[-1]
#ts_data.columns.str.split('__').str[-1]
ts_data

,TomorrowRiver:92743,TomorrowRiver:73262,TomorrowRiver:23983,TomorrowRiver:70548,TomorrowRiver:70262,TomorrowRiver:67401,TomorrowRiver:986,TomorrowRiver:68690,TomorrowRiver:23883,TomorrowRiver:72737,...,TomorrowRiver:67922,TomorrowRiver:4661,TomorrowRiver:92724,TomorrowRiver:602,TomorrowRiver:603,TomorrowRiver:68218,TomorrowRiver:68216,TomorrowRiver:68219,TomorrowRiver:90159,TomorrowRiver:93832
1,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00
2,3.149514e-31,3.083550e-13,2.663931e-95,0.000002,3.383515e-69,2.170843e-46,5.668807e-74,3.266619e-29,4.882848e-54,3.972549e-35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.877011e-19
3,1.920899e-20,2.549347e-08,3.389358e-81,0.000684,1.869941e-55,1.367037e-33,3.891717e-60,1.255159e-18,8.508016e-41,1.628349e-23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.749015e-11
4,2.028006e-15,1.419201e-06,5.195237e-73,0.005025,1.364082e-47,1.071487e-26,3.483969e-52,9.174176e-14,1.928208e-33,8.813122e-18,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.480974e-08
5,1.521443e-12,1.109408e-05,3.049297e-67,0.014037,3.907793e-42,3.186878e-22,1.225623e-46,5.261634e-11,1.661709e-28,2.366070e-14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.677526e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,3.966548e-03,1.153865e-03,4.245738e-02,0.035339,3.841655e-02,3.629232e-02,2.279774e-02,7.639249e-02,3.196826e-02,5.201480e-02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.543117e-02
1822,3.955246e-03,1.149971e-03,4.249338e-02,0.035216,3.841830e-02,3.622115e-02,2.280429e-02,7.614737e-02,3.192646e-02,5.187203e-02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.538145e-02
1823,3.944007e-03,1.146103e-03,4.252938e-02,0.035094,3.841981e-02,3.615018e-02,2.281073e-02,7.590379e-02,3.188463e-02,5.172999e-02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.533204e-02
1824,3.932829e-03,1.142261e-03,4.256539e-02,0.034973,3.842108e-02,3.607942e-02,2.281706e-02,7.566172e-02,3.184276e-02,5.158867e-02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.528295e-02


In [35]:
ts_path = template_dir / 'ts_obs.dat'
output_ts = [i.strip() for i in open(ts_path, 'r').readlines()]


In [36]:
ts_df = pd.DataFrame(index = ts_obs, data = {'obsname':ts_obs,'obs_values':np.nan})

In [38]:
for cob in ts_df.index:
    criv,ctime = cob.split('__')
    ts_df.loc[cob,'obs_values'] = ts_data.loc[int(ctime)][criv]

In [39]:
ts_df

,obsname,obs_values
TomorrowRiver:92696__1460,TomorrowRiver:92696__1460,0.000031
TomorrowRiver:92696__1461,TomorrowRiver:92696__1461,0.000031
TomorrowRiver:92696__1462,TomorrowRiver:92696__1462,0.000031
TomorrowRiver:92696__1463,TomorrowRiver:92696__1463,0.000033
TomorrowRiver:92696__1464,TomorrowRiver:92696__1464,0.000041
...,...,...
TomorrowRiver:70974__1821,TomorrowRiver:70974__1821,0.003353
TomorrowRiver:70974__1822,TomorrowRiver:70974__1822,0.003354
TomorrowRiver:70974__1823,TomorrowRiver:70974__1823,0.003355
TomorrowRiver:70974__1824,TomorrowRiver:70974__1824,0.003356


In [40]:
allout = pd.concat([bdplobs,ts_df])

In [41]:
allout['obs_values'].to_csv(template_dir / 'allobs.out', sep = ' ', header=None)

In [42]:
with open(template_dir / 'allobs.out.ins', 'w') as ofp:
    ofp.write('pif ~\n')
    [ofp.write(f'l1 w !{i}!\n') for i in allout.index]

# Make PST file

In [43]:
cwd = os.getcwd()

In [44]:
os.chdir(template_dir)
pst = pyemu.Pst.from_io_files(*pyemu.utils.parse_dir_for_io_files('.'))
os.chdir(cwd)

error parsing metadata from 'obsnme', continuing


In [45]:
pars = pst.parameter_data

In [47]:
pars.iloc[pars_df.index]['parval1'] = pars_df.parval1

C:\Users\pruita\Miniconda3\lib\site-packages\pandas\core\dtypes\cast.py:1642: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [48]:
pars.loc[pars.index.str.startswith('well_'),'parlbnd'] = \
        pars.loc[pars.index.str.startswith('well_'),'parval1']-.1
pars.loc[pars.index.str.startswith('well_'),'parubnd'] = \
        pars.loc[pars.index.str.startswith('well_'),'parval1']+.1


In [49]:
pars.loc[pars.parlbnd <=0, 'parlbnd'] = 0.01
pars.loc[pars.parubnd >=1, 'parubnd'] = 1
pars.partrans = 'none'

In [50]:
pars.loc['global_s', 'parlbnd'] = 0.05
pars.loc['global_s', 'parubnd'] = 0.2 # can make these a function of starting values later

pars.loc['global_t', 'parlbnd'] = 0.025 * pars.loc['global_t', 'parval1']
pars.loc['global_t', 'parubnd'] = 12 * pars.loc['global_t', 'parval1']
pars.loc['global_t', 'partrans'] = 'log' 
pars

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
global_s,global_s,none,factor,1.0,0.050,0.2,pargp,1.0,0.0,1
global_t,global_t,log,factor,1.0,0.025,12.0,pargp,1.0,0.0,1
well_1013__stream_apportionment121,well_1013__stream_apportionment121,none,factor,1.0,0.900,1.0,pargp,1.0,0.0,1
well_1302__stream_apportionment14,well_1302__stream_apportionment14,none,factor,1.0,0.900,1.0,pargp,1.0,0.0,1
well_1323__stream_apportionment48,well_1323__stream_apportionment48,none,factor,1.0,0.900,1.0,pargp,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...
well_92885__stream_apportionment92,well_92885__stream_apportionment92,none,factor,1.0,0.900,1.0,pargp,1.0,0.0,1
well_92981__stream_apportionment24,well_92981__stream_apportionment24,none,factor,1.0,0.900,1.0,pargp,1.0,0.0,1
well_93141__stream_apportionment78,well_93141__stream_apportionment78,none,factor,1.0,0.900,1.0,pargp,1.0,0.0,1
well_93832__stream_apportionment159,well_93832__stream_apportionment159,none,factor,1.0,0.900,1.0,pargp,1.0,0.0,1


In [51]:
pst.control_data.noptmax = -1
pst.model_command = [f'python run_pycap.py {input_yml} ts_obs.dat']
pst.pestpp_options['par_sigma_range'] = 6
pst.pestpp_options['ies_num_reals'] = 500


In [52]:
pst.write(str(template_dir / 'prior_mc.pst'), version=2)

noptmax:-1, npar_adj:206, nnz_obs:939


C:\Users\pruita\Miniconda3\lib\site-packages\pandas\core\dtypes\cast.py:1642: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
C:\Users\pruita\Miniconda3\lib\site-packages\pandas\core\dtypes\cast.py:1642: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [53]:
shutil.copy2('run_pycap.py', template_dir / 'run_pycap.py')
# shutil.copy2('../dependencies/bin/pestpp-ies', template_dir / 'pestpp-ies')
shutil.copytree('../hicap_analysis/', template_dir / 'hicap_analysis')
shutil.rmtree(template_dir / 'hicap_analysis' / 'tests')

In [56]:
if 'window' in platform.platform().lower():
    pestpp_ex = '../../dependencies/win_bin/pestpp-ies'
else:
    pestpp_ex = '../../dependencies/mac_bin/pestpp-ies'
pyemu.os_utils.start_workers(
        worker_dir=str(template_dir), exe_rel_path=pestpp_ex,
        pst_rel_path='prior_mc.pst', num_workers=None,
        worker_root='./', master_dir='MASTER')

Exception: start_workers() master returned non-zero: 3221226505